In [1]:
import pandas as pd

In [2]:
df_metadata = pd.read_csv('../attribute_extraction_metadata_template/Initial Attribute Definition for First Release - UPDATED SHEET .csv')

In [3]:
df_metadata = df_metadata[['category', 'attribute_field', 'category_attributevalue', 'description', 'max_multi_select', 'usage', 'entry mode']]

In [4]:
metadata_category = set(df_metadata.category)

In [5]:
category2config = {}
for i in metadata_category:
    df_tmp = df_metadata[df_metadata.category == i]
    category2config[i] = sorted(df_tmp.to_dict('records'), key=lambda x: x['attribute_field'])

In [6]:
df_samples = pd.read_csv('allstratified_sample_4806_query.csv').rename(columns={'top_query_classification_taxonomy': 'category_path'})

In [7]:
df_samples = df_samples[df_samples.category_path.apply(lambda x: any([i.startswith(x) for i in metadata_category]))]
df_samples = df_samples.sample(10, random_state=42)

In [8]:
df_samples

,label_ordering,query,sample_method,category_path
652,13434,tachos de basura,uniform,Home & Garden > Household Cleaning > Household...
2253,44644,robe hiver femme,head,Women's Clothing > Sweaters > Dresses
3839,79080,teck deck fingerboards,head,Toys & Hobbies
4119,84546,etagere salle de bain,head,Home Improvement > Bathroom Fixtures > Bathroo...
946,17674,cornici foto a muro,head,Home & Garden > Home Decor > Frames
2621,52701,dinossauros de brinquedo,head,Toys & Hobbies > Action & Toy Figures > Animal...
577,12139,pañuelos de algodon,uniform,Apparel Accessories > Men's Ties & Handkerchiefs
4180,85989,one piece mug,uniform,"Home & Garden > Kitchen,Dining & Bar > Drinkwa..."
650,13404,tenis para mujeres,head,Sports > Racquet Sports > Tennis > Tennis Shorts
4358,89949,school bags for kids,head,Luggage & Bags > Kids & Baby's Bags > School Bags


In [9]:
from functools import reduce

In [10]:
reduce(lambda x,y: x + y, *[[], [2]])

[2]

In [16]:
from copy import deepcopy
import hashlib
from collections import OrderedDict
import json
def sortOD(od):
    res = OrderedDict()
    for k, v in sorted(od.items()):
        if isinstance(v, dict):
            res[k] = sortOD(v)
        else:
            res[k] = deepcopy(v)
    return res

In [19]:
res = []
for i in df_samples[['query', 'sample_method', 'category_path']].to_dict('records'):
    i['text'] = f"Query: {i['query']}"
    attr_config = []
    for j in metadata_category:
        if j.startswith(i['category_path']):
            attr_config += deepcopy(category2config[j])
    i['attribute_config'] = attr_config
    task_json = json.dumps(sortOD(i)).encode('utf-8')
    task_id = hashlib.md5(task_json).hexdigest()
    i['task_id'] = f"query_attribution_md_{task_id}"
    i['preloading_result'] = None
    res.append(i)

In [27]:
pd.DataFrame(res).to_json('example_offshore_input_query_attr_extract_v2.json', lines=True, orient='records')